In [1]:
# !pip install safetensors \
#     nvidia-pyindex nvidia-tensorrt \
#     tritonclient[http]

In [2]:
# !pip install onnx

In [5]:
# !pip install optimum

In [6]:
from optimum.onnxruntime import ORTModelForSequenceClassification,ORTModel


In [7]:
# --- Common Utilities and Setup ---
import os
import json
import torch
import transformers
import accelerate
import huggingface_hub
import peft
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from peft import LoraConfig, get_peft_model, TaskType
from sklearn.preprocessing import LabelEncoder
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
from sklearn.utils import resample
from collections import Counter
import time
import onnxruntime as ort
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import List
import pickle

print("peft:", peft.__version__)
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("Accelerate:", accelerate.__version__)
print("Huggingface Hub:", huggingface_hub.__version__)

# Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

peft: 0.14.0
Torch: 2.2.2
Transformers: 4.49.0
Accelerate: 1.4.0
Huggingface Hub: 0.29.1


In [8]:
def update_model_dict(model_alias, MODEL_NAME):
    if not os.path.exists('model_dict.json'):
        model_dict = {}
    else:
        with open('model_dict.json', 'r') as file:
            model_dict = json.load(file)

    model_dict[model_alias] = MODEL_NAME

    with open('model_dict.json', 'w') as file:
        json.dump(model_dict, file)

In [9]:
def load_and_preprocess_data(filepath="./data/train-00000-of-00001-a5a7c6e4bb30b016.parquet"):
    """Loads and preprocesses the dataset."""
    df = pd.read_parquet(filepath)
    df = df[['conversation', 'issue_area']]
    print("Original distribution:\n", df['issue_area'].value_counts())
    label_encoder = LabelEncoder()
    df["labels"] = label_encoder.fit_transform(df["issue_area"])

    #saving Label-encoder
    label_encoder_path = f"model-metric/{model_alias}/label_encoder.pkl"
    os.makedirs(os.path.dirname(label_encoder_path), exist_ok=True)
    with open(label_encoder_path, "wb") as f:
        pickle.dump(label_encoder, f)
        
    return df, label_encoder

In [10]:
def balance_dataset(df, max_count=100, random_state=42):
    """Balances the dataset using oversampling."""
    balanced_df = pd.DataFrame()
    for issue in df['issue_area'].unique():
        subset = df[df['issue_area'] == issue]
        balanced_subset = resample(subset, replace=True, n_samples=max_count, random_state=random_state)
        balanced_df = pd.concat([balanced_df, balanced_subset])
    return balanced_df.sample(frac=1, random_state=random_state).reset_index(drop=True)


In [11]:
def preprocess_conversation(conversation):
    """Preprocesses a conversation."""
    if isinstance(conversation, list):
        return " ".join([turn.get('text', '') for turn in conversation if isinstance(turn, dict)])
    return str(conversation).lower()

In [12]:
# Define PyTorch Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length=512):
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index]
        inputs = self.tokenizer(
            row["conversation"],
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt"
        )
        input_ids = inputs["input_ids"].squeeze(0)
        attention_mask = inputs["attention_mask"].squeeze(0)
        label = torch.tensor(row["labels"], dtype=torch.long)
        return input_ids, attention_mask, label

In [13]:
def create_dataloaders(df, tokenizer, batch_size=8, train_ratio=0.75):
    """Creates train and test DataLoaders."""
    train_size = int(train_ratio * len(df))
    train_df, test_df = df[:train_size], df[train_size:]
    train_dataset = CustomDataset(train_df, tokenizer)
    test_dataset = CustomDataset(test_df, tokenizer)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader, test_df

In [14]:
class DistilBERTWithLoRA(nn.Module):
    def __init__(self, num_labels, lora_r=8, lora_alpha=16, lora_dropout=0.1):
        super(DistilBERTWithLoRA, self).__init__()
        # Load the base model with the correct number of labels
        self.bert = AutoModelForSequenceClassification.from_pretrained(
            "distilbert/distilbert-base-uncased",
            num_labels=num_labels  # Ensure this matches the number of classes
        )
        
        # LoRA Configuration
        lora_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=lora_r,
            lora_alpha=lora_alpha,
            lora_dropout=lora_dropout,
            target_modules=["q_lin", "k_lin", "v_lin"]
        )
        self.bert = get_peft_model(self.bert, lora_config)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits  # Return the logits directly

In [32]:
class DistilBERTWithLoRAOptimized(nn.Module):
    def __init__(self, num_labels, lora_r=8, lora_alpha=16, lora_dropout=0.1):
        super(DistilBERTWithLoRAOptimized, self).__init__()
        # Load the base model with the correct number of labels
        # onnx_model = ORTModelForSequenceClassification.from_pretrained('distilbert_onnx', provider='CUDAExecutionProvider')
        self.onnx_model = ORTModelForSequenceClassification.from_pretrained(
            "distilbert_onnx",
            num_labels=num_labels  # Ensure this matches the number of classes
        )
        
        # # LoRA Configuration
        # lora_config = LoraConfig(
        #     task_type=TaskType.SEQ_CLS,
        #     r=lora_r,
        #     lora_alpha=lora_alpha,
        #     lora_dropout=lora_dropout,
        #     target_modules=["q_lin", "k_lin", "v_lin"]
        # )
        # self.onnx_model = get_peft_model(self.onnx_model, lora_config)

    def forward(self, input_ids, attention_mask):
        outputs = self.onnx_model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.logits  # Return the logits directly

In [16]:
# Function to compute class weights
def compute_class_weights(labels, num_classes):
    counter = Counter(labels)
    total_samples = len(labels)
    weights = [total_samples / (num_classes * counter[i]) for i in range(num_classes)]
    return torch.tensor(weights, dtype=torch.float)

In [17]:
def train_model(model, train_loader, model_alias, epochs=3, learning_rate=5e-5, class_weights=None):
    """Trains the model and saves logs, metrics, and model weights."""
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.train()

    # Create directory for storing model metrics
    model_dir = f"model-metric/{model_alias}"
    os.makedirs(model_dir, exist_ok=True)

    # TensorBoard writer in the model directory
    writer = SummaryWriter(log_dir=model_dir)

    # Set up loss function and optimizer
    criterion = torch.nn.CrossEntropyLoss(weight=class_weights.to(device) if class_weights is not None else None)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

    epoch_losses = []
    metrics_data = []

    for epoch in range(epochs):
        start_time = time.time()
        total_loss = 0
        all_preds, all_labels = [], []

        for batch_idx, batch in enumerate(train_loader):
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            preds = torch.argmax(outputs, dim=1).cpu().tolist()
            labels = labels.cpu().tolist()
            all_preds.extend(preds)
            all_labels.extend(labels)

            # Log batch loss every 10 batches
            if batch_idx % 10 == 0:
                writer.add_scalar("BatchLoss/train", loss.item(), epoch * len(train_loader) + batch_idx)

        # Compute epoch metrics
        avg_loss = total_loss / len(train_loader)
        epoch_losses.append(avg_loss)
        accuracy = accuracy_score(all_labels, all_preds)
        precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
        epoch_time = time.time() - start_time

        # Store metrics for CSV logging
        metrics_data.append([epoch + 1, avg_loss, accuracy, precision, recall, f1, epoch_time])

        # Print metrics
        print(f"Epoch {epoch+1}: Loss={avg_loss:.4f}, Accuracy={accuracy:.4f}, Precision={precision:.4f}, Recall={recall:.4f}, F1-score={f1:.4f}, Time={epoch_time:.2f}s")

        # Log metrics to TensorBoard
        writer.add_scalar("Loss/train", avg_loss, epoch)
        writer.add_scalar("Accuracy/train", accuracy, epoch)
        writer.add_scalar("Precision/train", precision, epoch)
        writer.add_scalar("Recall/train", recall, epoch)
        writer.add_scalar("F1-score/train", f1, epoch)
        writer.add_scalar("Time/Epoch", epoch_time, epoch)

    # Save model KPIs as CSV
    metrics_df = pd.DataFrame(metrics_data, columns=["Epoch", "Loss", "Accuracy", "Precision", "Recall", "F1-score", "Time (s)"])
    metrics_df.to_csv(os.path.join(model_dir, "training_metrics.csv"), index=False)

    # Save training loss curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, epochs + 1), epoch_losses, marker='o', linestyle='-', color='b')
    plt.title('Training Loss Over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.grid(True)
    loss_plot_path = os.path.join(model_dir, "training_loss.png")
    plt.savefig(loss_plot_path)
    writer.add_figure("Training Loss", plt.gcf(), close=True)

    # Save model weights
    model_path = os.path.join(model_dir, f"{model_alias}.pth")
    torch.save(model.state_dict(), model_path)

    writer.flush()
    writer.close()


In [18]:
import os
import time
import torch
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
from sklearn.metrics import classification_report, confusion_matrix, precision_recall_fscore_support

def evaluate_model(model, test_loader, label_encoder, model_alias):
    """Evaluates the model and saves metrics, logs, and confusion matrix."""

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # Create directory for storing model metrics
    model_dir = f"model-metric/{model_alias}"
    os.makedirs(model_dir, exist_ok=True)

    # Initialize TensorBoard writer
    writer = SummaryWriter(log_dir=model_dir)

    all_preds, all_labels = [], []
    start_time = time.time()

    with torch.no_grad():
        for batch in test_loader:
            input_ids, attention_mask, labels = [x.to(device) for x in batch]
            outputs = model(input_ids, attention_mask=attention_mask)
            preds = torch.argmax(outputs, dim=1).cpu().tolist()
            labels = labels.cpu().tolist()
            all_preds.extend(preds)
            all_labels.extend(labels)

    eval_time = time.time() - start_time
    class_names = label_encoder.classes_

    # Compute metrics
    precision, recall, f1, support = precision_recall_fscore_support(all_labels, all_preds, average=None)
    class_metrics = pd.DataFrame({
        'Class': class_names,
        'Precision': precision,
        'Recall': recall,
        'F1-Score': f1,
        'Support': support
    })

    overall_precision, overall_recall, overall_f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')

    # Print and save classification report
    print("\nClassification Report:\n", classification_report(all_labels, all_preds, target_names=class_names))

    # Generate confusion matrix
    cm = confusion_matrix(all_labels, all_preds)
    cm_df = pd.DataFrame(cm, index=class_names, columns=class_names)

    # Plot confusion matrix
    plt.figure(figsize=(12, 10))
    sns.heatmap(cm_df, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix')
    plt.tight_layout()

    # Save confusion matrix plot
    confusion_matrix_path = os.path.join(model_dir, "confusion_matrix.png")
    plt.savefig(confusion_matrix_path)
    writer.add_figure("Confusion Matrix", plt.gcf(), close=True)

    # Print overall metrics
    print("\nPer-class Metrics:\n", class_metrics.to_string(index=False))
    print(f"\nOverall Metrics:\nPrecision: {overall_precision:.4f}, Recall: {overall_recall:.4f}, F1-score: {overall_f1:.4f}, Eval Time: {eval_time:.2f}s")

    # Log metrics to TensorBoard
    writer.add_scalar("Precision/test", overall_precision)
    writer.add_scalar("Recall/test", overall_recall)
    writer.add_scalar("F1-score/test", overall_f1)
    writer.add_scalar("Evaluation Time", eval_time)

    # Log per-class metrics
    for i, class_name in enumerate(class_names):
        writer.add_scalar(f"Precision/{class_name}", precision[i])
        writer.add_scalar(f"Recall/{class_name}", recall[i])
        writer.add_scalar(f"F1-score/{class_name}", f1[i])

    writer.flush()
    writer.close()

    # Save evaluation metrics
    class_metrics.to_csv(os.path.join(model_dir, "class_metrics.csv"), index=False)
    cm_df.to_csv(os.path.join(model_dir, "confusion_matrix.csv"))

    return class_metrics, cm_df


In [19]:
import os
import time
import torch
import numpy as np
import onnxruntime as ort
import pandas as pd
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from torch.utils.tensorboard import SummaryWriter

def export_to_onnx(model, tokenizer, model_alias):
    """Exports the model to ONNX format."""
    model.eval().to("cpu")
    sample_input = tokenizer("test", return_tensors="pt")
    input_names = ["input_ids", "attention_mask"]
    output_names = ["output"]
    
    model_dir = f"model-metric/{model_alias}"
    os.makedirs(model_dir, exist_ok=True)
    onnx_path = os.path.join(model_dir, f"{model_alias}.onnx")
    
    torch.onnx.export(
        model, 
        (sample_input["input_ids"], sample_input["attention_mask"]), 
        onnx_path, 
        input_names=input_names, 
        output_names=output_names, 
        dynamic_axes={
            "input_ids": {0: "batch", 1: "sequence"}, 
            "attention_mask": {0: "batch", 1: "sequence"}, 
            "output": {0: "batch"}
        }
    )
    print(f"ONNX model exported to {onnx_path}")
    return onnx_path

def run_onnx_inference(onnx_path, input_ids, attention_mask):
    """Runs inference using ONNX Runtime."""
    ort_session = ort.InferenceSession(onnx_path)
    ort_inputs = {"input_ids": input_ids.tolist(), "attention_mask": attention_mask.tolist()}
    ort_outputs = ort_session.run(None, ort_inputs)
    return torch.tensor(np.array(ort_outputs[0]), dtype=torch.float32)

def compare_inference_performance(model, tokenizer, test_df, label_encoder, model_alias):
    """Compares inference performance between PyTorch and ONNX Runtime."""
    model_dir = f"model-metric/{model_alias}"
    os.makedirs(model_dir, exist_ok=True)
    writer = SummaryWriter(log_dir=model_dir)
    
    sample_batch = test_df.sample(50)
    test_dataset_batch = CustomDataset(sample_batch, tokenizer)
    test_loader_batch = DataLoader(test_dataset_batch, batch_size=len(sample_batch), shuffle=False)
    batch = next(iter(test_loader_batch))
    input_ids, attention_mask, labels = batch
    
    # PyTorch Inference
    model.eval().to(device)
    start_time_torch = time.time()
    with torch.no_grad():
        torch_outputs = model(input_ids.to(device), attention_mask.to(device)).cpu()
    latency_torch = time.time() - start_time_torch
    throughput_torch = len(sample_batch) / latency_torch
    
    # ONNX Inference
    onnx_path = export_to_onnx(model, tokenizer, model_alias)
    start_time_onnx = time.time()
    onnx_outputs = run_onnx_inference(onnx_path, input_ids, attention_mask)
    latency_onnx = time.time() - start_time_onnx
    throughput_onnx = len(sample_batch) / latency_onnx
    
    print(f"PyTorch Inference - Latency: {latency_torch:.4f}s, Throughput: {throughput_torch:.2f} samples/s")
    print(f"ONNX Inference - Latency: {latency_onnx:.4f}s, Throughput: {throughput_onnx:.2f} samples/s")
    
    torch_preds = torch.argmax(torch_outputs, dim=1).tolist()
    onnx_preds = torch.argmax(onnx_outputs, dim=1).tolist()
    actual_labels = labels.tolist()
    
    # Compare predictions
    torch_report = classification_report(actual_labels, torch_preds, target_names=label_encoder.classes_, output_dict=True)
    onnx_report = classification_report(actual_labels, onnx_preds, target_names=label_encoder.classes_, output_dict=True)
    
    torch_df = pd.DataFrame(torch_report).transpose()
    onnx_df = pd.DataFrame(onnx_report).transpose()
    
    torch_df.to_csv(os.path.join(model_dir, "torch_classification_report.csv"))
    onnx_df.to_csv(os.path.join(model_dir, "onnx_classification_report.csv"))
    
    # Log metrics to TensorBoard
    writer.add_scalar("Latency/PyTorch", latency_torch)
    writer.add_scalar("Throughput/PyTorch", throughput_torch)
    writer.add_scalar("Latency/ONNX", latency_onnx)
    writer.add_scalar("Throughput/ONNX", throughput_onnx)
    
    writer.flush()
    writer.close()
    
    return torch_df, onnx_df


In [20]:
def compute_class_weights(labels, num_classes):
    counter = Counter(labels)
    total_samples = len(labels)
    weights = [total_samples / (num_classes * counter[i]) for i in range(num_classes)]
    return torch.tensor(weights, dtype=torch.float)

In [21]:
MODEL_NAME = "distilbert/distilbert-base-uncased"
model_alias = 'distilbert-cased-lora'
update_model_dict(model_alias, MODEL_NAME)

In [22]:
df, label_encoder = load_and_preprocess_data()
balanced_df = balance_dataset(df)
balanced_df['conversation'] = balanced_df['conversation'].apply(preprocess_conversation)

Original distribution:
 issue_area
Cancellations and returns    286
Order                        270
Login and Account            151
Shopping                     116
Warranty                     105
Shipping                      72
Name: count, dtype: int64


In [23]:
    # Tokenization and DataLoaders
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilbert-base-uncased")
train_loader, test_loader, test_df = create_dataloaders(balanced_df, tokenizer)

In [24]:
# Model Initialization and Training
num_classes = len(label_encoder.classes_)
model = DistilBERTWithLoRA(num_labels=num_classes)
class_weights = compute_class_weights(balanced_df['labels'], num_classes)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/manmehro1/Desktop/repository/lora-training-explaination/verizon-poc/.venv/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [25]:
train_model(model, train_loader,model_alias=model_alias, epochs=10, learning_rate=5e-5, class_weights=class_weights)

Epoch 1: Loss=1.7884, Accuracy=0.1578, Precision=0.2058, Recall=0.1578, F1-score=0.1273, Time=211.36s
Epoch 2: Loss=1.7386, Accuracy=0.3422, Precision=0.4862, Recall=0.3422, F1-score=0.2944, Time=193.56s
Epoch 3: Loss=1.4201, Accuracy=0.6622, Precision=0.6998, Recall=0.6622, F1-score=0.6377, Time=206.09s
Epoch 4: Loss=0.8198, Accuracy=0.7911, Precision=0.7908, Recall=0.7911, F1-score=0.7829, Time=201.69s
Epoch 5: Loss=0.5616, Accuracy=0.8444, Precision=0.8413, Recall=0.8444, F1-score=0.8393, Time=205.10s
Epoch 6: Loss=0.4478, Accuracy=0.8644, Precision=0.8660, Recall=0.8644, F1-score=0.8605, Time=204.40s
Epoch 7: Loss=0.3736, Accuracy=0.8978, Precision=0.8984, Recall=0.8978, F1-score=0.8958, Time=205.96s
Epoch 8: Loss=0.3048, Accuracy=0.9156, Precision=0.9152, Recall=0.9156, F1-score=0.9140, Time=206.25s
Epoch 9: Loss=0.2541, Accuracy=0.9200, Precision=0.9225, Recall=0.9200, F1-score=0.9185, Time=218.22s
Epoch 10: Loss=0.2185, Accuracy=0.9333, Precision=0.9341, Recall=0.9333, F1-score=

In [26]:
# Model Evaluation
evaluate_model(model, test_loader, label_encoder, model_alias)


Classification Report:
                            precision    recall  f1-score   support

Cancellations and returns       0.79      0.95      0.86        20
        Login and Account       1.00      0.93      0.96        28
                    Order       0.89      0.80      0.84        30
                 Shipping       0.85      1.00      0.92        23
                 Shopping       1.00      0.86      0.93        22
                 Warranty       1.00      1.00      1.00        27

                 accuracy                           0.92       150
                macro avg       0.92      0.92      0.92       150
             weighted avg       0.93      0.92      0.92       150


Per-class Metrics:
                     Class  Precision   Recall  F1-Score  Support
Cancellations and returns   0.791667 0.950000  0.863636       20
        Login and Account   1.000000 0.928571  0.962963       28
                    Order   0.888889 0.800000  0.842105       30
                 Ship

(                       Class  Precision    Recall  F1-Score  Support
 0  Cancellations and returns   0.791667  0.950000  0.863636       20
 1          Login and Account   1.000000  0.928571  0.962963       28
 2                      Order   0.888889  0.800000  0.842105       30
 3                   Shipping   0.851852  1.000000  0.920000       23
 4                   Shopping   1.000000  0.863636  0.926829       22
 5                   Warranty   1.000000  1.000000  1.000000       27,
                            Cancellations and returns  Login and Account  \
 Cancellations and returns                         19                  0   
 Login and Account                                  1                 26   
 Order                                              4                  0   
 Shipping                                           0                  0   
 Shopping                                           0                  0   
 Warranty                                           0

In [33]:
optimum_model = DistilBERTWithLoRAOptimized(num_labels=num_classes)

RepositoryNotFoundError: 404 Client Error. (Request ID: Root=1-68087185-448c963115a6779b7a73a334;ee22e074-8517-47bf-b300-2dfefe6c118c)

Repository Not Found for url: https://huggingface.co/api/models/distilbert_onnx/tree/main?recursive=True&expand=False.
Please make sure you specified the correct `repo_id` and `repo_type`.
If you are trying to access a private or gated repo, make sure you are authenticated.

In [ ]:
train_model(optimum_model, train_loader,model_alias=model_alias, epochs=10, learning_rate=5e-5, class_weights=class_weights)

In [ ]:
# Model Evaluation
evaluate_model(model, test_loader, label_encoder, model_alias)

In [27]:
tokenizer.save_pretrained(f"model-metric/{model_alias}/tokenizer/")

('model-metric/distilbert-cased/tokenizer/tokenizer_config.json',
 'model-metric/distilbert-cased/tokenizer/special_tokens_map.json',
 'model-metric/distilbert-cased/tokenizer/vocab.txt',
 'model-metric/distilbert-cased/tokenizer/added_tokens.json',
 'model-metric/distilbert-cased/tokenizer/tokenizer.json')